# DES Agent

## 1. Imports

In [1]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from fastmcp import Client
import asyncio
import json

## 2. Test code

## Agent pipeline

In [6]:
async def fetch_schema():
    """Fetch experiment schema using FastMCP client"""
    async with Client("http://localhost:8000") as client:
        resource = await client.call_tools("get_experiment_template")
        return json.loads(resource.content.text)

In [7]:
async def run_simulation(parameters):
    """Run simulation using FastMCP client"""
    async with Client("http://localhost:8000") as client:
        result = await client.call_tool("run_experiment", parameters)
        return result.content[0].text

In [8]:
async def main(model_name="gemma3:latest"):
    # 1. Connect to Ollama model
    llm = OllamaLLM(model=model_name, base_url="http://localhost:11434")
    
    # 2. Fetch experiment schema using FastMCP
    schema = await fetch_schema()
    print("Schema retrieved:", schema)
    
    # 3. Create LangChain prompt
    PROMPT_TEMPLATE = """
    You are a discrete-event simulation assistant. 
    
    You help turn natural language into structured simulation parameters.
    
    Here is the simulation configuration schema:
    {schema}
    
    User request:
    {user_input}
    
    Return the parameters as a JSON object.
    """
    
    prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
    chain = prompt | llm
    
    # 4. Example usage
    user_input = "Run with 14 operators and 12 nurses"
    response = chain.invoke({"schema": schema, "user_input": user_input})
    
    # 5. Parse and run simulation
    parameters = json.loads(response)
    result = await run_simulation(parameters)
    print("Simulation result:", result)

In [9]:
# use await in Jupyter book asyncio.run(main()) in module
await main()

RuntimeError: Client failed to connect: Session terminated

In [ ]:
# agent_pipeline.py
# 1. Connect to running Ollama model (gemma3:latest)
llm = OllamaLLM(model="gemma3:latest", base_url="http://localhost:11434")

# 2. Fetch experiment schema for prompting
schema = requests.get("http://localhost:8000/resource/experiment_template")
print(schema)

schema = schema.json()

# 3. Create LangChain prompt
PROMPT_TEMPLATE = """
You are a discrete-event simulation assistant. 

You help turn natural language into structured simulation parameters.

Here is the simulation configuration schema:
{schema}

User request:
{user_input}

Return the parameters as a JSON object.
"""

prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
chain = prompt | llm

In [ ]:
schema

In [ ]:
# 4. Get user request
user_input = input("Describe your experiment (e.g. 'Run with 14 operators and add 5% demand'): ")

# 5. LLM turns prompt into JSON params
param_response = chain.invoke({
    "schema": str(schema),  # convert dict to string for prompt clarity
    "user_input": user_input
})

print("\n🧠 LLM parameter output:")
print(param_response)

In [ ]:
# 6. Submit to MCP server and get results
try:
    params = eval(param_response.strip())  # optionally use `json.loads` if well-formatted
    results = requests.post(
        "http://localhost:8000/tool/run_experiment",
        json={"parameters": params}
    ).json()
    
    print("\n📊 Simulation Results:")
    for k, v in results.items():
        print(f" - {k}: {v:.2f}")
except Exception as e:
    print("❌ Could not parse or run experiment:", e)